# Transforming with LLMs

**Summary**
This notebook demonstrates how to use LLMs to transform text in various ways, including translation, tone adjustment, format conversion, and grammar correction.

**Key Takeaways:**
1.  **Translation**: Translate text between languages, including detecting the source language automatically.
2.  **Universal Translator**: Process multiple messages in different languages in a loop for efficient batch translation.
3.  **Tone Transformation**: Convert casual/slang text to formal business language and vice versa.
4.  **Format Conversion**: Transform data between formats (e.g., Python dict/JSON to HTML tables).
5.  **Grammar & Spelling Correction**: Proofread and correct text, including handling homonyms and common errors.
6.  **Diff Visualization**: Use libraries like `redlines` to visualize changes between original and corrected text.

---


In [16]:
# Loading env variables from .env file
import os

from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
client = OpenAI()


def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,  # This is the degree of randomness of the output
    )
    return response.choices[0].message.content

Translation


In [3]:
prompt = """
Translate the following English text to Spanish:
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría ordenar una licuadora.


In [4]:
prompt = """
Tell me which language this is:
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This is French.


In [5]:
prompt = """
Translate the following  text to French and Spanish
and English pirate:
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: Je veux commander un ballon de basket
Spanish: Quiero ordenar un balón de baloncesto
Pirate: I be wantin' to order a basketball


In [7]:
prompt = """
Translate the following text to Spanish in both the
formal and informal forms:
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


Universal translator


In [ ]:
user_messages = [
    # System performance is slower than normal
    "La performance du système est plus lente que d'habitude.",
    # My monitor has pixels that are not lighting
    "Mi monitor tiene píxeles que no se iluminan.",
    # My mouse is not working
    "Il mio mouse non funziona",
    # My keyboard has a broken control key
    "Mój klawisz Ctrl jest zepsuty",
    "我的屏幕在闪烁"                                               # My screen is flashing
]

In [9]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (French): La performance du système est plus lente que d'habitude.
English: "The system performance is slower than usual."

Korean: "시스템 성능이 평소보다 느립니다." 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."
Korean: "내 모니터에는 불이 켜지지 않는 픽셀이 있습니다." 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Korean: 제 Ctrl 키가 고장 났어요 

Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering
Korean: 내 화면이 깜박거립니다 



Tone transformation


In [10]:
prompt = """
Translate the following from slang to a business letter:
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the specifications of a standing lamp that I believe may be of interest to you. 

Sincerely,
Joe


Format conversion


In [12]:
data_json = {"resturant employees": [
    {"name": "Shyam", "email": "shyamjaiswal@gmail.com"},
    {"name": "Bob", "email": "bob32@gmail.com"},
    {"name": "Jai", "email": "jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<html>
<head>
  <title>Restaurant Employees</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </table>
</body>
</html>


In [13]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


Grammar checking


In [ ]:
text = [
    # The girl has a ball.
    "The girl with the black and white puppies have a ball.",
    "Yolanda has her notebook.",  # ok
    "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
    "Your going to need you’re notebook.",  # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?",  # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found
Their goes my freedom. There going to bring they’re suitcases.

No errors found.

Rewritten: 
Their goes my freedom. There going to bring their suitcases.
You're going to need your notebook.
No errors found.
No errors found


In [26]:
text = """
Got this for my daughter for her birthday cuz she keeps taking
mine from my room.  Yes, adults also like pandas too.  She takes
it everywhere with her, and it's super soft and cute.  One of the
ears is a bit lower than the other, and I don't think that was
designed to be asymmetrical. It's a bit small for what I paid for it
though. I think there might be other options that are bigger for
the same price.  It arrived a day earlier than expected, so I got
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [27]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>mine from my room.  Yes, adults also like pandas too.  She takes <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>it everywhere with her, and it's super soft and cute.  One of the <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>ears is a bit lower than the other, and I don't think that was <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>designed to be asymmetrical. It's a bit small for what I paid for it <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>though. I think there might be other options that are bigger for <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>the same price.  It arrived a day earlier than expected, so I got <span style='color:red;font-weight:700;text-decoration:line-through;'>¶ </span>to play with it myself before I gave it to my daughter.

In [23]:
prompt = f"""
proofread and correct this review. Make it more compelling.
Ensure it follows APA style guide and targets an advanced reader.
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

# Review of Panda Plush Toy

I purchased this adorable panda plush toy for my daughter's birthday after she kept taking mine from my room. Contrary to popular belief, adults can also appreciate the charm of pandas. The plush toy is not only soft and cute, but my daughter carries it everywhere with her.

However, upon closer inspection, I noticed that one of the ears is slightly lower than the other, which seems to be an unintentional design flaw. Additionally, I found the size of the plush toy to be smaller than expected given its price point. I believe there are larger options available for the same cost.

Despite these minor drawbacks, I was pleasantly surprised when the plush toy arrived a day earlier than anticipated. This allowed me to enjoy playing with it myself before presenting it to my daughter. Overall, while there are some room for improvements, the quality and cuteness of the panda plush toy make it a worthwhile purchase for any panda enthusiast, regardless of age.